In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

In [2]:
df = pd.read_csv(r"D:\flood prediction\imputated weather dataset.csv")
df.head()

,Station_Names,Year,Month,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Wind_Speed,Cloud_Coverage,Bright_Sunshine,Station_Number,X_COR,Y_COR,LATITUDE,LONGITUDE,ALT,Period,Flood?
0,Barisal,1949.0,1.0,29.4,12.3,0.0,68.0,0.453704,0.6,7.831915,41950.0,536809.8,510151.9,22.7,90.36,4.0,1949-01,0.0
1,Barisal,1949.0,2.0,33.9,15.2,9.0,63.0,0.659259,0.9,8.314894,41950.0,536809.8,510151.9,22.7,90.36,4.0,1949-02,0.0
2,Barisal,1949.0,3.0,36.7,20.2,8.0,59.0,1.085185,1.5,8.131915,41950.0,536809.8,510151.9,22.7,90.36,4.0,1949-03,0.0
3,Barisal,1949.0,4.0,33.9,23.9,140.0,71.0,1.772222,3.9,8.219149,41950.0,536809.8,510151.9,22.7,90.36,4.0,1949-04,0.0
4,Barisal,1949.0,5.0,35.6,25.0,217.0,76.0,1.703704,4.1,7.046809,41950.0,536809.8,510151.9,22.7,90.36,4.0,1949-05,0.0


In [3]:
x = df.drop(['Station_Names','Rainfall', 'Flood?', 'Period'], axis = 1)
y = df['Rainfall']

In [4]:
rf = RandomForestRegressor(n_jobs=-1, random_state=42)

In [5]:
boruta_selector = BorutaPy(
    estimator=rf,
    n_estimators='auto',
    verbose=0,  # Set to 0 to suppress output during fit
    random_state=42
)

In [6]:
boruta_selector.fit(x.values, y.values)

BorutaPy(estimator=RandomForestRegressor(n_estimators=46, n_jobs=-1,
                                         random_state=RandomState(MT19937) at 0x27ED4E5E740),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x27ED4E5E740)

In [7]:
# important features
significant_features = x.columns[boruta_selector.support_].tolist()
significant_features

['Year',
 'Month',
 'Max_Temp',
 'Min_Temp',
 'Relative_Humidity',
 'Wind_Speed',
 'Cloud_Coverage',
 'Bright_Sunshine',
 'Station_Number',
 'LONGITUDE']

#### The features above are statistically significant

## Lasso

In [8]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

In [9]:
# Split data (optional step, helps to prevent overfitting on small datasets)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [10]:
# Fit Lasso model
lasso = Lasso(alpha=1) 
lasso.fit(x_train, y_train)

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.066e+06, tolerance: 1.119e+05
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=1)

In [11]:
selected_features = np.where(lasso.coef_ != 0)[0]
selected_features

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13], dtype=int64)

In [12]:
lasso.coef_

array([-3.01836673e-01, -1.76367133e+00,  2.67032529e+00, -3.12886984e+00,
        3.20347497e+00,  9.84307201e+00,  6.28248921e+01, -3.31556529e+01,
       -2.02910395e+00,  4.75078101e-04, -6.01021596e-04, -0.00000000e+00,
       -6.99927766e+00,  8.63505194e-02])

In [13]:
# cross validation
from sklearn.linear_model import LassoCV

In [14]:
lasso_cv = LassoCV(cv=5).fit(x_train, y_train)

In [15]:
selected_features_cv = np.where(lasso_cv.coef_ != 0)[0]
selected_features_cv

array([ 9, 10], dtype=int64)

###### After cross validation bright_sunshine and station_number become significant in L1 regularization. As we can see Lasso is not effective in this case.

## Ridge 

In [16]:
from sklearn.linear_model import Ridge

In [17]:
# Fit Ridge model
ridge = Ridge(alpha=1.0)
ridge.fit(x_train, y_train)

Ridge()

In [18]:
# Set a threshold to select important features
threshold = 0.1
selected_features_ridge = np.where(abs(ridge.coef_) > threshold)[0]
selected_features_ridge

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 11, 12, 13], dtype=int64)

In [19]:
# cross validation
from sklearn.linear_model import RidgeCV

In [20]:
ridge_cv = RidgeCV(cv=5).fit(x_train, y_train)

In [21]:
selected_features_ridgecv = np.where(lasso_cv.coef_ != 0)[0]
selected_features_ridgecv

array([ 9, 10], dtype=int64)

##### After cross validation bright_sunshine and station_number become significant in Ridge Regression. As  we can see, Ridege is not that effective in this case.